In [46]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [47]:
import pandas as pd
import json

# Load the Excel file
file_path = 'cleaned_file.xlsx'
df = pd.read_excel(file_path, usecols=['Test Scenario'])

# Initialize variables
use_case_name = ''
scenarios = []
result = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    test_scenario = row['Test Scenario']
    if test_scenario.endswith('.txt'):
        # If we already have a use case name, save the previous use case and its scenarios
        if use_case_name:
            result.append({
                'use_case_name': use_case_name,
                'scenarios': scenarios
            })
        # Set the new use case name and reset the scenarios list
        use_case_name = test_scenario
        scenarios = []
    else:
        # Add the scenario to the current use case's scenarios list
        scenarios.append(test_scenario)

# Add the last use case and its scenarios
if use_case_name:
    result.append({
        'use_case_name': use_case_name,
        'scenarios': scenarios
    })

# Convert the result to a JSON string

In [48]:
def find_scenarios_by_use_case_name(result_json, use_case_name):
    # Deserialize the JSON string back into a Python list
    use_case_name = use_case_name +".txt"
    for item in result_json:
        if item['use_case_name'] == use_case_name:
            return item['scenarios']
    
    # Return None if the use case name is not found
    return None

In [49]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [50]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [51]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [52]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [53]:
use_case_list = [
"Remove the book.txt",
"Remove the author.txt",
"Remove the genre.txt"]


In [54]:
BASELINE_GENERATE_TC = """
read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":{
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
  },
}
If there are more than one test case for this scenario, continue writing other test case in this form."""

In [55]:
use_case_directory_path = r"D:\GPT-testing\SpecificationData\Book(Github)"
test_scenario_directory_path = r"D:\Dissertation-GPT\evaluate\baseline 2\book\ts temp"
save_testcase_path = r"D:\Dissertation-GPT\evaluate\baseline 2\book\tc temp"

In [56]:

for usecase in use_case_list:
  base_name = os.path.basename(usecase)
  usecase_name = os.path.splitext(base_name)[0]
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  testscenario_path = os.path.join (test_scenario_directory_path,f"{usecase_name}.txt" )
  print(testscenario_path)
  testscenario_list = read_file_to_list(testscenario_path)
  # testscenario_list = find_scenarios_by_use_case_name(result,usecase_name)
  print(testscenario_list)
  for testscenario in testscenario_list:
    content = "Test scenarios:\n"+ testscenario
    content += "\nUse case:" + read_file_content(usecase_path)
    print(content)
    promptExtractCondition = [
      { "role": "system", "content": BASELINE_GENERATE_TC},
      { "role": "user", "content": content}
    ]
    gpt_response = askJSON(promptExtractCondition, client, model)
    print(gpt_response)
    print()
    json_data = json.loads(gpt_response)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
    testcase_path = os.path.join (save_testcase_path,f"{usecase_name}-{testscenario}.txt" )
    write_json_to_txt_raw(json_data,testcase_path )
  print("f{usecase_name} gen test case complete")

D:\Dissertation-GPT\evaluate\baseline 2\book\ts temp\Remove the book.txt
['4. DeleteBook_NavigateAwayWithoutDeletion_BooksMenu', '5. DeleteBook_NavigateAwayWithoutDeletion_GenresMenu', '6. DeleteBook_NavigateAwayWithoutDeletion_AuthorsMenu', '7. DeleteBook_NavigateAwayWithoutDeletion_HomeMenu', '8. DeleteBook_NavigateAwayWithoutDeletion_APIMenu']
Test scenarios:
4. DeleteBook_NavigateAwayWithoutDeletion_BooksMenu
Use case:Remove the book

Main scenario:
User clicks the "Delete" button (to the right from the book he/she wants to delete) on the book list/details page;
Application displays a dialog page to confirm destroying the record for chosen book;
User confirms deletion by pressing "Confirm" button.

Cancel operation scenario:
User clicks the "Delete" button (to the right from the book he/she wants to delete) on the book list/details page;
Application displays a dialog page to confirm destroying the record for chosen book;
User presses “Cancel” button;
Data doesn’t delete in the data